In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,0,Pevek,RU,69.7008,170.3133,-22.92,100,95,5.48,overcast clouds,2022-02-05 00:01:30
1,1,Eureka,US,40.8021,-124.1637,60.67,81,97,5.01,overcast clouds,2022-02-05 00:01:05
2,2,Lebu,CL,-37.6167,-73.6500,58.35,85,2,16.78,clear sky,2022-02-05 00:06:27
3,3,New Norfolk,AU,-42.7826,147.0587,65.75,59,45,1.01,scattered clouds,2022-02-05 00:02:57
4,4,Rikitea,PF,-23.1203,-134.9692,79.02,67,5,5.97,clear sky,2022-02-05 00:05:31


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
trip_locations_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                    (city_data_df["Max Temp"] >= min_temp)]
trip_locations_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
4,4,Rikitea,PF,-23.1203,-134.9692,79.02,67,5,5.97,clear sky,2022-02-05 00:05:31
8,8,Butaritari,KI,3.0707,172.7902,82.58,76,13,19.95,few clouds,2022-02-05 00:05:28
9,9,Bathsheba,BB,13.2167,-59.5167,77.61,69,20,10.36,few clouds,2022-02-05 00:05:32
13,13,Touros,BR,-5.1989,-35.4608,80.64,72,11,12.93,few clouds,2022-02-05 00:06:29
14,14,Atuona,PF,-9.8000,-139.0333,78.40,77,25,16.58,scattered clouds,2022-02-05 00:06:30


In [5]:
# 4a. Determine if there are any empty rows.
trip_locations_df.count()

City_ID                133
City                   133
Country                133
Lat                    133
Lng                    133
Max Temp               133
Humidity               133
Cloudiness             133
Wind Speed             133
Current Description    133
Date                   133
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_data_df= trip_locations_df.dropna(subset = ['Country'])
clean_data_df.reset_index(drop=True)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Date
0,4,Rikitea,PF,-23.1203,-134.9692,79.02,67,5,5.97,clear sky,2022-02-05 00:05:31
1,8,Butaritari,KI,3.0707,172.7902,82.58,76,13,19.95,few clouds,2022-02-05 00:05:28
2,9,Bathsheba,BB,13.2167,-59.5167,77.61,69,20,10.36,few clouds,2022-02-05 00:05:32
3,13,Touros,BR,-5.1989,-35.4608,80.64,72,11,12.93,few clouds,2022-02-05 00:06:29
4,14,Atuona,PF,-9.8000,-139.0333,78.40,77,25,16.58,scattered clouds,2022-02-05 00:06:30
...,...,...,...,...,...,...,...,...,...,...,...
128,525,Piranhas,BR,-16.4269,-51.8222,75.83,82,100,2.06,overcast clouds,2022-02-05 00:20:04
129,526,Caravelas,BR,-17.7125,-39.2481,76.93,86,100,8.23,overcast clouds,2022-02-05 00:20:04
130,528,Kavaratti,IN,10.5669,72.6420,77.70,59,93,4.56,overcast clouds,2022-02-05 00:20:04
131,534,Ca Mau,VN,9.1769,105.1500,77.90,71,59,11.86,broken clouds,2022-02-05 00:20:07


In [7]:
clean_data_df.count()

City_ID                133
City                   133
Country                133
Lat                    133
Lng                    133
Max Temp               133
Humidity               133
Cloudiness             133
Wind Speed             133
Current Description    133
Date                   133
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_data_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Rikitea,PF,79.02,clear sky,-23.1203,-134.9692,
8,Butaritari,KI,82.58,few clouds,3.0707,172.7902,
9,Bathsheba,BB,77.61,few clouds,13.2167,-59.5167,
13,Touros,BR,80.64,few clouds,-5.1989,-35.4608,
14,Atuona,PF,78.40,scattered clouds,-9.8000,-139.0333,
15,Takoradi,GH,79.30,scattered clouds,4.8845,-1.7554,
17,Pandan,PH,76.46,overcast clouds,14.0453,124.1694,
24,Vaini,TO,78.53,moderate rain,-21.2000,-175.2000,
30,Mtwara,TZ,76.53,light rain,-10.2667,40.1833,
34,Saint-Philippe,RE,80.15,light rain,-21.3585,55.7679,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    params["location"] =f"{lat},{lng}"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError): print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
hotel_df.count()

City                   133
Country                133
Max Temp               133
Current Description    133
Lat                    133
Lng                    133
Hotel Name             133
dtype: int64

In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(subset=["Hotel Name"])
clean_hotel_df.reset_index(drop=True)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,79.02,clear sky,-23.1203,-134.9692,People ThankYou
1,Butaritari,KI,82.58,few clouds,3.0707,172.7902,Isles Sunset Lodge
2,Bathsheba,BB,77.61,few clouds,13.2167,-59.5167,Atlantis Hotel
3,Touros,BR,80.64,few clouds,-5.1989,-35.4608,INN NEW HORIZON
4,Atuona,PF,78.40,scattered clouds,-9.8000,-139.0333,Villa Enata
...,...,...,...,...,...,...,...
128,Piranhas,BR,75.83,overcast clouds,-16.4269,-51.8222,Planalto Hotel
129,Caravelas,BR,76.93,overcast clouds,-17.7125,-39.2481,Pousada dos Navegantes
130,Kavaratti,IN,77.70,overcast clouds,10.5669,72.6420,Hotels in Lakshadweep Islands
131,Ca Mau,VN,77.90,broken clouds,9.1769,105.1500,Khách Sạn Mường Thanh Luxury Cà Mau


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt<dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
                                 max_intensity=300,point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))